# Write a function to calculate gini index for the given dataset

x: list of counts
<br>
if we have 30 elements from one class, 40 elements from the second and 50 elements from third then x == [30,40,50]

In [1]:
import numpy as np

In [2]:
def gini(x:list):
    P=[]
    for i in range(len(x)):
        p=x[i]/sum(x)
        P.append(p)
    p_sq=[j**2  for j in P]
    Gin=1-sum(p_sq)
    return Gin

In [3]:
x=[30,40,50]
gini(x)

0.6527777777777778

# Write a function to calculate gini index for the given split

x: list of counts for the left dataset 
<br>
y: list of counts for the right dataset

In [4]:
def split_gini(x:list, y:list):
    x_p=[]
    y_p=[]
    for i in range(len(x)):
        x_p.append((x[i]/sum(x))**2)
        y_p.append((y[i]/sum(y))**2)
    GI=1-(sum(x_p)+sum(y_p))/2
    return GI

In [5]:
x=[25,50,60]
y=[35,33,80]
split_gini(x,y)

0.6165872454035617

# Apply Decision Tree Classifier on the 'classification.csv' dataset from the previous homework.

In [101]:
cd C:\Users\Dell\Desktop\Machine Learning\Programming

C:\Users\Dell\Desktop\Machine Learning\Programming


In [297]:
import pandas as pd
df=pd.read_csv("Class 10/classification.csv")
df.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,college degree,17,12,176,9.3,11.359392,5.008608,1
1,27,no high school,10,6,31,17.3,1.362202,4.000798,0
2,40,no high school,15,14,55,5.5,0.856075,2.168925,0
3,41,no high school,15,14,120,2.9,2.658720,0.821280,0
4,24,high school,2,0,28,17.3,1.787436,3.056564,1


In [298]:
y=df['default']

In [299]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image

In [300]:
df = df.replace({'college degree':'graduate','undergraduate':'graduate','postgraduate': 'graduate'})
df=pd.concat([df,pd.get_dummies(df['ed'])], axis=1)
df=df.drop(['default','ed'], axis=1)
df.head()

,age,employ,address,income,debtinc,creddebt,othdebt,graduate,high school,no high school
0,41,17,12,176,9.3,11.359392,5.008608,1,0,0
1,27,10,6,31,17.3,1.362202,4.000798,0,0,1
2,40,15,14,55,5.5,0.856075,2.168925,0,0,1
3,41,15,14,120,2.9,2.658720,0.821280,0,0,1
4,24,2,0,28,17.3,1.787436,3.056564,0,1,0


In [339]:
df.describe()

,age,employ,address,income,debtinc,creddebt,othdebt,graduate,high school,no high school
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,34.860000,8.388571,8.278571,45.601429,10.260571,1.553553,3.058209,0.185714,0.282857,0.531429
std,7.997342,6.658039,6.824877,36.814226,6.827234,2.117197,3.287555,0.389154,0.450710,0.499368
min,20.000000,0.000000,0.000000,14.000000,0.400000,0.011696,0.045584,0.000000,0.000000,0.000000
25%,29.000000,3.000000,3.000000,24.000000,5.000000,0.369059,1.044178,0.000000,0.000000,0.000000
50%,34.000000,7.000000,7.000000,34.000000,8.600000,0.854869,1.987567,0.000000,0.000000,1.000000
75%,40.000000,12.000000,12.000000,55.000000,14.125000,1.901955,3.923065,0.000000,1.000000,1.000000
max,56.000000,31.000000,34.000000,446.000000,41.300000,20.561310,27.033600,1.000000,1.000000,1.000000


In [307]:
def depth(model, dt, y, folds=10):
    kf = KFold(n_splits=folds)
    test_rmse = []
    train_rmse = []
    depthlist=np.array(range(1,20))
    
    for idep in depthlist:

        for train_index, test_index in kf.split(dt):
            xTrain, xTest = dt.loc[train_index], dt.loc[test_index]
            yTrain, yTest = y[train_index], y[test_index]

            model=DecisionTreeClassifier(max_depth=idep)
            model.fit(xTrain, yTrain)
            predict_test = model.predict(xTest)
            predict_train = model.predict(xTrain)

            test_rmse.append(np.sqrt(mean_squared_error(yTest, predict_test)))
            train_rmse.append(np.sqrt(mean_squared_error(yTrain, predict_train)))
            
        print("Depth:", idep, "MEAN TEST RMSE:", np.mean(test_rmse), "MEAN TRAIN RMSE:", np.mean(train_rmse))


In [309]:
depth(dtc,df[:int(0.8*len(df))],y[:int(0.8*len(y))])

Depth: 1 MEAN TEST RMSE: 0.5096127452135631 MEAN TRAIN RMSE: 0.5050811643852191
Depth: 2 MEAN TEST RMSE: 0.5029704256487142 MEAN TRAIN RMSE: 0.4853571558709551
Depth: 3 MEAN TEST RMSE: 0.4987055770134849 MEAN TRAIN RMSE: 0.46933812560972954
Depth: 4 MEAN TEST RMSE: 0.49836311939259625 MEAN TRAIN RMSE: 0.45518390603147374
Depth: 5 MEAN TEST RMSE: 0.49754885472007937 MEAN TRAIN RMSE: 0.4408944929147195
Depth: 6 MEAN TEST RMSE: 0.49840093353894804 MEAN TRAIN RMSE: 0.4238537661945344
Depth: 7 MEAN TEST RMSE: 0.49888878869711856 MEAN TRAIN RMSE: 0.4051078749204026
Depth: 8 MEAN TEST RMSE: 0.4986261145681598 MEAN TRAIN RMSE: 0.3848817234880335
Depth: 9 MEAN TEST RMSE: 0.5002273322089034 MEAN TRAIN RMSE: 0.3639598718102532
Depth: 10 MEAN TEST RMSE: 0.5023074711879988 MEAN TRAIN RMSE: 0.3417108442385576
Depth: 11 MEAN TEST RMSE: 0.5038242337447546 MEAN TRAIN RMSE: 0.3199709918639707
Depth: 12 MEAN TEST RMSE: 0.5055414014624239 MEAN TRAIN RMSE: 0.299295277550603
Depth: 13 MEAN TEST RMSE: 0.5059

In [337]:
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(df[:int(0.8*len(df))],y[:int(0.8*len(y))])
pred = clf.predict(df[int(0.8*len(df)):])
np.sqrt(mean_squared_error(y[int(0.8*len(df)):], pred))

0.49280538030458115

# Implement Random Forest

In [446]:
import random

In [447]:
class RandomForest:
    def __init__(self):
        clf=decision
    def fit(train_df,train_y, n_trees, n_bootstrap, n_features, dt_max_depth):
        forest=[]

        for i in range(n_trees):
            bootstrap_indices=np_random.randint(0, len(train_df),n_bootstrap)
            df_bootstrapped=train_df.iloc[bootstrap_indices]
            y_bootstrapped=train_y.iloc[bootstrap_indices]
            tree = DecisionTreeClassifier(dt_max_depth).fit(train_df, train_y)
            forest.append(tree)
    
        return forest
        
    def predict(test_df, forest):
        tr_pred=clf.predict(test_df)
        ts_pred=clf.predict
        return random_forest_predictions
    
    def score(predictions, test_y):
        accuracy=clf.score(, test_df)
        return accuracy

IndentationError: expected an indented block (<ipython-input-447-6ad7121cec10>, line 4)